### Backtest Results Verification

This notebook performs a manual, step-by-step calculation of portfolio returns for a **single, specific date** and compares them against the results generated by the automated backtesting engine (`py9`).

Its purpose is to serve as a sanity check and a debugging tool to ensure the core logic of the backtester is correct.

**Workflow:**
1.  **Setup:** Define the single `VERIFICATION_DATE_STR` to be checked.
2.  **Load Data:** Load the three required files: the portfolio selection for the target date, the historical price data, and the master backtest results file.
3.  **Manual Calculation:** Manually identify the buy/sell dates and calculate the portfolio returns for each weighting scheme.
4.  **Compare & Verify:** Extract the corresponding results from the master backtest file, display them side-by-side with the manual calculations, and assert that they are numerically equal.

### Setup and Configuration

**This is the only cell you need to edit.** Set the `VERIFICATION_DATE_STR` to the date of the selection run you want to verify.

In [ ]:
# py10_backtest_verification.ipynb

import sys
from pathlib import Path
import pandas as pd
import numpy as np


# --- THIS IS THE ONLY PARAMETER TO CHANGE ---
# VERIFICATION_DATE_STR = "2025-06-11" 

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()

# Corrected Logic:
# The notebook is at ROOT/notebooks/_working.
# To get to the ROOT directory, we need to go up two levels.
# NOTEBOOK_DIR.parent -> .../notebooks
# NOTEBOOK_DIR.parent.parent -> .../stocks (This is the correct ROOT_DIR)
ROOT_DIR = NOTEBOOK_DIR.parent.parent

SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

sys.path.append(str(ROOT_DIR / 'notebooks'))  # Add config.py to sys.path

# --- Verification (Optional, but good for debugging) ---
print(f"Current Notebook Dir: {NOTEBOOK_DIR}")
print(f"Calculated ROOT_DIR:   {ROOT_DIR}")
print(f"Calculated SRC_DIR:    {SRC_DIR}")
print(f"sys.path contains SRC_DIR: {str(SRC_DIR) in sys.path}")

# --- Local Imports ---
import utils
from config import DATE_STR, RISK_FREE_RATE_DAILY

# --- THIS IS THE ONLY PARAMETER TO CHANGE ---
VERIFICATION_DATE_STR = DATE_STR

# --- File Path Construction (using our standard principles) ---
# We derive all paths from the verification date
SELECTION_DIR = ROOT_DIR / 'output' / 'selection_results'
BACKTEST_DIR = ROOT_DIR / 'output' / 'backtest_results'
DATA_DIR = ROOT_DIR / 'data' # Assuming data dir is at root/data


# Construct the exact filenames we expect
SELECTION_FILE_PATH = SELECTION_DIR / f"{VERIFICATION_DATE_STR}_short_term_mean_reversion.parquet"
BACKTEST_RESULTS_PATH = BACKTEST_DIR / 'backtest_master_results.parquet'
HISTORICAL_PRICES_PATH = DATA_DIR / 'df_adj_close.parquet'

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.6f}'.format)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Verifying backtest for selection date: {VERIFICATION_DATE_STR}")
print(f"Selection File: {SELECTION_FILE_PATH}")
print(f"Backtest Results: {BACKTEST_RESULTS_PATH}")
print(f"Price Data: {HISTORICAL_PRICES_PATH}")

### Step 1: Load All Required Data

In [ ]:
print("--- Step 1: Loading all required data files ---")

try:
    # Load the specific portfolio selection for the verification date
    df_selection = pd.read_parquet(SELECTION_FILE_PATH)
    print(f"✅ Successfully loaded selection file for {VERIFICATION_DATE_STR}.")
    
    # Load the master backtest results file
    df_backtest = pd.read_parquet(BACKTEST_RESULTS_PATH)
    print("✅ Successfully loaded master backtest results.")

    # Load the historical price data
    df_prices = pd.read_parquet(HISTORICAL_PRICES_PATH)
    df_prices.index = pd.to_datetime(df_prices.index)
    print("✅ Successfully loaded historical price data.")
    
    data_loaded_successfully = True

except FileNotFoundError as e:
    print(f"❌ ERROR: Could not find a required file. {e}")
    data_loaded_successfully = False

### Step 2: Manual Performance Calculation

In [ ]:
# In py10_backtest_verification.ipynb, replace the "Step 2" cell with this:

if data_loaded_successfully:
    print(f"\n--- Step 2: Manually calculating performance for {VERIFICATION_DATE_STR} ---")
    
    # Isolate the tickers from our portfolio
    tickers = df_selection.index.to_list()
    
    # --- Correct Date Logic ---
    # Use get_indexer(), which supports the 'method' argument, to find the position.
    date_loc = df_prices.index.get_indexer([pd.to_datetime(VERIFICATION_DATE_STR)], method='ffill')[0]
    
    # Get the next two trading days from the index
    buy_date = df_prices.index[date_loc + 1]
    sell_date = df_prices.index[date_loc + 2]
    
    print(f"Selection Date (actual used): {df_prices.index[date_loc].date()}")
    print(f"Buy Date (T+1): {buy_date.date()}")
    print(f"Sell Date (T+2): {sell_date.date()}")
    
    # Extract the prices for the buy and sell dates for our selected tickers
    buy_prices = df_prices.loc[buy_date, tickers]
    sell_prices = df_prices.loc[sell_date, tickers]
    
    # Calculate individual returns
    individual_returns = (sell_prices - buy_prices) / buy_prices
    
    # --- 1. Manual Calculation of Weighted Portfolio Returns ---
    weights_df = df_selection[['Weight_EW', 'Weight_IV', 'Weight_SW']]
    weighted_returns = weights_df.multiply(individual_returns, axis=0)
    manual_portfolio_results = weighted_returns.sum()
    manual_portfolio_results.name = "manual_return"
    manual_portfolio_results.index = manual_portfolio_results.index.str.split('_').str[-1] # Align index
    
    # --- 2. Manual Calculation of Scheme-Independent Metrics ---
    manual_std_dev = individual_returns.std()
    manual_avg_return = individual_returns.mean()
    if manual_std_dev > 1e-9:
        manual_sharpe = (manual_avg_return - RISK_FREE_RATE_DAILY) / manual_std_dev
    else:
        manual_sharpe = np.nan

    # --- Display All Manual Results ---
    print("\n--- Manual Calculation: Portfolio Returns ---")
    print("Meaning: The final return of the portfolio, calculated as the sum of (individual_stock_return * weight).")
    display(manual_portfolio_results.to_frame())
    
    print("\n--- Manual Calculation: Risk/Reward Metrics ---")
    print("Meaning (std_dev_return): The standard deviation of the individual stock returns. Measures component volatility.")
    print("Meaning (sharpe_ratio_period): (Average Individual Stock Return - Risk-Free Rate) / std_dev_return. Measures risk-adjusted return of the average stock.")
    print(f"  - Std Dev of Returns: {manual_std_dev:.6f}")
    print(f"  - Sharpe Ratio:       {manual_sharpe:.6f}")

### Step 3: Compare and Verify

In [ ]:
# In py10_backtest_verification.ipynb, replace the "Step 3" cell with this:

if data_loaded_successfully:
    print(f"\n--- Step 3: Comparing manual results with automated backtest results ---")
    
    # --- 1. Verify Weighted Portfolio Returns ---
    print("\n--- Verification 1: Portfolio Returns (Scheme-Dependent) ---")
    print("These values depend on the weighting scheme (EW, IV, SW).")
    
    # Extract the results from the automated backtest file
    automated_results_df = df_backtest[
        (df_backtest['selection_date'] == VERIFICATION_DATE_STR)
    ].set_index('scheme')

    # Create a comparison DataFrame for portfolio returns
    df_port_comparison = pd.concat([manual_portfolio_results, automated_results_df['portfolio_return']], axis=1)
    df_port_comparison.columns = ['manual_return', 'backtest_return']
    df_port_comparison['match'] = np.isclose(df_port_comparison['manual_return'], df_port_comparison['backtest_return'])

    print("Comparison Table:")
    display(df_port_comparison)
    
    assert df_port_comparison['match'].all(), "❌ VERIFICATION FAILED: Manual and backtest portfolio returns do not match!"
    print("✅ Portfolio Return Verification Successful.")

    # --- 2. Verify Scheme-Independent Metrics ---
    print("\n--- Verification 2: Risk/Reward Metrics (Scheme-Independent) ---")
    print("These values are based only on the set of selected stocks, not their weights, so they are identical across schemes for a given day.")
    
    # These metrics are the same for all schemes on a given day, so we can take from the first row
    automated_std_dev = automated_results_df['std_dev_return'].iloc[0]
    automated_sharpe = automated_results_df['sharpe_ratio_period'].iloc[0]
    
    # Create a new comparison DataFrame for these metrics
    metrics_data = {
        'Metric': ['std_dev_return', 'sharpe_ratio_period'],
        'Manual_Value': [manual_std_dev, manual_sharpe],
        'Backtest_Value': [automated_std_dev, automated_sharpe]
    }
    df_metrics_comparison = pd.DataFrame(metrics_data).set_index('Metric')
    df_metrics_comparison['match'] = np.isclose(df_metrics_comparison['Manual_Value'], df_metrics_comparison['Backtest_Value'], equal_nan=True)

    print("Comparison Table:")
    display(df_metrics_comparison)
    
    assert df_metrics_comparison['match'].all(), "❌ VERIFICATION FAILED: Manual and backtest risk metrics do not match!"
    print("✅ Risk/Reward Metrics Verification Successful.")
    
    print("\n\n✅✅✅ OVERALL VERIFICATION SUCCESSFUL ✅✅✅")

In [ ]:
print(f'df_backtest:\n{df_backtest.head()}')